<a href="https://colab.research.google.com/github/God-Orcale/AI_Test/blob/main/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5,3,requires_grad=True)
b = torch.randn(3,requires_grad= True)
z = torch.matmul(x,w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)
# 反向传播。在此算法中，参数根据loss函数的梯度进行调整添加到给定的参数中。
# 计算梯度，PyTorch有一个内置的微分引擎叫torch.autograd。
# 它支持自动计算任何计算图。
print(f"x:{x},\n w:{w},\n b:{b},\n z:{z},\n loss:{loss}")

x:tensor([1., 1., 1., 1., 1.]),
 w:tensor([[-0.5307, -0.7968, -0.2264],
        [ 0.1109, -0.1617,  0.1858],
        [-0.8023, -0.0873, -0.1678],
        [ 0.7206, -1.1387, -0.4571],
        [ 1.4859,  1.2503,  0.4301]], requires_grad=True),
 b:tensor([-1.1817, -0.3948, -0.1673], requires_grad=True),
 z:tensor([-0.1974, -1.3290, -0.4026], grad_fn=<AddBackward0>),
 loss:0.4487166106700897


In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")
# 应用张量以构建计算图的函数实际上是此类对象知道
# 如何计算正向的函数，以及如何计算它在向后传播步骤中的导数。

Gradient function for z = <AddBackward0 object at 0x78a5884ef5b0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x78a5884eeec0>


In [3]:
loss.backward()
print(w.grad)
print(b.grad)
# 计算我们的损失函数关于参数的导数
# 为了计算这些导数，我们调用loss.backward()，然后从w.grad和b.grad中检索值

tensor([[0.0405, 0.2233, 0.0685],
        [0.0405, 0.2233, 0.0685],
        [0.0405, 0.2233, 0.0685],
        [0.0405, 0.2233, 0.0685],
        [0.0405, 0.2233, 0.0685]])
tensor([0.0405, 0.2233, 0.0685])


In [4]:
z = torch.matmul(x,w)+b
print(z.requires_grad)
with torch.no_grad():
  z = torch.matmul(x,w)+b
print(z.requires_grad)
# 当我们有训练模型，只想将其应用于一些输入数据，即我们只想通过网络进行前向计算。
# 我们可以通过使用torch.no_grad()Block包围我们的计算代码来停止跟踪计算

True
False


In [5]:
z = torch.matmul(x,w)+b
z_det = z.detach()
print(z_det.requires_grad)
# 实现相同结果的另一种方法是使用detach()在 Tensor 上

False


禁用渐变跟踪：

将神经网络中的某些参数标记为冻结参数。

在执行仅前向传递时加快计算速度，因为执行不跟踪渐变会更有效。

In [6]:
inp = torch.eye(4,5,requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out),retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out),retain_graph=True)
print(f"\nSeconf call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out),retain_graph=True)
print(f"\n Call after zeroing gradients\n{inp.grad}")
# 我们有一个标量损失函数，我们需要计算相对于某些参数的梯度。
# 但是，也有情况当output函数是任意张量时。
# 在本例中，PyTorch允许您计算所谓的雅可比积，而不是实际的梯度。

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Seconf call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

 Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


注意，当我们使用相同的参数，则gradient的值不同。发生这种情况是因为在执行传播时，PyTorch 会累积gradients，即计算出的gradients的值被添加到计算图的所有叶节点的属性中。如果要计算正确的梯度，需要先将property归零。